In [1]:
import pandas as pd
import numpy as np
from typing import Iterable
from sklearn.preprocessing import LabelEncoder

In [2]:
def universal_one_hot_encoder(df: pd.DataFrame, target_col: str):
    data_to_encode = df.copy()

    # Определяем категориальные признаки (все, что не числа и не целевая колонка)
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    categorical_features = [
        col for col in data_to_encode.columns if col not in numeric_cols or col == target_col
    ]
    if target_col in categorical_features:
        categorical_features.remove(target_col)

    # Применяем One-Hot Encoding
    data_ohe = pd.get_dummies(
        data_to_encode,
        columns=categorical_features,
        dummy_na=False,
        dtype=int
    )

    # Собираем список всех дискретных колонок для CTGAN
    # Это все колонки, которых не было в исходном списке числовых колонок
    discrete_features_ohe = [
        col for col in data_ohe.columns if col not in numeric_cols or col == target_col
    ]

    return data_ohe, discrete_features_ohe

In [3]:
def universal_drop_nans(df: pd.DataFrame, patterns: Iterable[str] | str | None = None, regex: bool = False) -> pd.DataFrame:
    """
    Заменяет указанные паттерны на NaN и возвращает новую таблицу с удалёнными строками, содержащими NaN.
    - patterns: строка или итерация строк; если None, просто выполняется dropna().
    - regex: если True, паттерны трактуются как регулярные выражения.
    """
    df_copy = df.copy()
    if patterns is None:
        return df_copy.dropna()
    if isinstance(patterns, str):
        patterns = [patterns]
    # replace поддерживает список значений; используем параметр regex при необходимости
    df_copy.replace(to_replace=list(patterns), value=np.nan, inplace=True, regex=regex)
    return df_copy.dropna()

In [4]:
def universal_label_encoder(df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    df_copy = df.copy()
    le = LabelEncoder()
    df_copy[target_col] = le.fit_transform(df_copy[target_col].astype(str))
    return df_copy

In [5]:
data = pd.read_csv('../data/unprocessed/Iris.csv')
target_col = 'Species'
data = universal_drop_nans(data)
data, discrete_features_ohe = universal_one_hot_encoder(data, target_col)
data = universal_label_encoder(data, target_col)
data.to_csv('../data/processed/Iris_processed.csv', index=False)

In [6]:
data

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,2
146,147,6.3,2.5,5.0,1.9,2
147,148,6.5,3.0,5.2,2.0,2
148,149,6.2,3.4,5.4,2.3,2


In [7]:
discrete_features_ohe

['Species']